In [1]:
import os
os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [ ]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 456 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

def objective_function_2(trial):

    params = {
        'topK': trial.suggest_int('topK', 5, 1000),
        'l1_ratio': trial.suggest_float('l1_ratio', 1e-5, 1, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 1),
    }


    MAP = 0
    recommender = SLIMElasticNetRecommender(URM_train)
    recommender.fit(**params)

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [20]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Slimnet_firstrun_noKFOLD.csv", index = False)

In [21]:
import optuna

optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_2,
                      callbacks=[save_results],
                      n_trials = 100)

[I 2024-11-28 17:51:38,424] A new study created in memory with name: no-name-b9f6ea64-a152-464e-b0d6-b66f404372c5


SLIMElasticNetRecommender: Processed 13082 (34.3%) in 5.00 min. Items per second: 43.60
SLIMElasticNetRecommender: Processed 26193 (68.7%) in 10.00 min. Items per second: 43.65
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 14.58 min. Items per second: 43.58
EvaluatorHoldout: Processed 35280 (100.0%) in 19.32 sec. Users per second: 1826


[I 2024-11-28 18:06:32,550] Trial 0 finished with value: 0.018272710605406292 and parameters: {'topK': 425, 'l1_ratio': 6.796499604599277e-05, 'alpha': 0.6436682568690753}. Best is trial 0 with value: 0.018272710605406292.


SLIMElasticNetRecommender: Processed 9298 (24.4%) in 5.00 min. Items per second: 30.99
SLIMElasticNetRecommender: Processed 18590 (48.8%) in 10.00 min. Items per second: 30.98
SLIMElasticNetRecommender: Processed 28546 (74.9%) in 15.00 min. Items per second: 31.71
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 19.60 min. Items per second: 32.42
EvaluatorHoldout: Processed 35280 (100.0%) in 24.04 sec. Users per second: 1467


[I 2024-11-28 18:26:32,562] Trial 1 finished with value: 0.02098395826584583 and parameters: {'topK': 606, 'l1_ratio': 4.626701411465976e-05, 'alpha': 0.11393574353005559}. Best is trial 1 with value: 0.02098395826584583.


...

SLIMElasticNetRecommender: Processed 9186 (24.1%) in 5.00 min. Items per second: 30.61
SLIMElasticNetRecommender: Processed 19235 (50.5%) in 10.00 min. Items per second: 32.05
SLIMElasticNetRecommender: Processed 29817 (78.2%) in 15.00 min. Items per second: 33.13
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 18.80 min. Items per second: 33.79
EvaluatorHoldout: Processed 35280 (100.0%) in 17.08 sec. Users per second: 2065


[I 2024-11-29 11:54:22,678] Trial 63 finished with value: 0.024194451193175166 and parameters: {'topK': 207, 'l1_ratio': 5.145461791468963e-05, 'alpha': 0.043238609053526104}. Best is trial 35 with value: 0.0325512712360066.


SLIMElasticNetRecommender: Processed 8218 (21.6%) in 5.00 min. Items per second: 27.39
SLIMElasticNetRecommender: Processed 13717 (36.0%) in 10.00 min. Items per second: 22.86
SLIMElasticNetRecommender: Processed 18258 (47.9%) in 15.00 min. Items per second: 20.28
SLIMElasticNetRecommender: Processed 22482 (59.0%) in 20.00 min. Items per second: 18.73
SLIMElasticNetRecommender: Processed 27137 (71.2%) in 25.00 min. Items per second: 18.09
SLIMElasticNetRecommender: Processed 32253 (84.6%) in 30.00 min. Items per second: 17.92
SLIMElasticNetRecommender: Processed 37693 (98.9%) in 35.00 min. Items per second: 17.95
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 35.42 min. Items per second: 17.94
EvaluatorHoldout: Processed 35280 (100.0%) in 38.74 sec. Users per second: 911


[I 2024-11-29 12:30:26,652] Trial 64 finished with value: 0.03135957914911889 and parameters: {'topK': 311, 'l1_ratio': 0.00022114961130715192, 'alpha': 0.002872530335753392}. Best is trial 35 with value: 0.0325512712360066.
[W 2024-11-29 12:32:07,427] Trial 65 failed with parameters: {'topK': 351, 'l1_ratio': 0.00038686272459315373, 'alpha': 0.06940118198228047} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-19-3af5e999a324>", line 14, in objective_function_2
    recommender.fit(**params)
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/sklearn/utils/_testing.py", line 156, in wrapper
    return fn(*args, **kwargs)
  File "/Users/lucapagano/Developer/RecSys/Competizione/Recommenders/SLIM/SLIMElasticNetRecommender.py", line 87, in 

KeyboardInterrupt: 

In [23]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.0325512712360066, 'topK': 734.0, 'l1_ratio': 0.00012146265227371838, 'alpha': 0.0014665049349635376}


In [35]:
# Let's retrain the model with the best hyperparameters and evaluate its accuracy on the test set
alpha = best_hyperparams["alpha"]
l1_ratio = best_hyperparams["l1_ratio"]
topK = int(best_hyperparams["topK"])
print("Best alpha: {}, l1_ratio: {}, topK: {}".format(alpha, l1_ratio, topK))

newRecommender = SLIMElasticNetRecommender(URM_train_validation)

Best alpha: 0.0014665049349635376, l1_ratio: 0.00012146265227371838, topK: 734


In [36]:
newRecommender.fit(topK=topK, l1_ratio=l1_ratio, alpha=alpha)

SLIMElasticNetRecommender: Processed 4017 (10.5%) in 5.00 min. Items per second: 13.39
SLIMElasticNetRecommender: Processed 8049 (21.1%) in 10.00 min. Items per second: 13.41
SLIMElasticNetRecommender: Processed 11994 (31.5%) in 15.00 min. Items per second: 13.32
SLIMElasticNetRecommender: Processed 16066 (42.1%) in 20.00 min. Items per second: 13.39
SLIMElasticNetRecommender: Processed 20110 (52.8%) in 25.00 min. Items per second: 13.40
SLIMElasticNetRecommender: Processed 23821 (62.5%) in 30.00 min. Items per second: 13.23
SLIMElasticNetRecommender: Processed 27413 (71.9%) in 35.00 min. Items per second: 13.05
SLIMElasticNetRecommender: Processed 31027 (81.4%) in 40.01 min. Items per second: 12.93
SLIMElasticNetRecommender: Processed 34774 (91.2%) in 45.01 min. Items per second: 12.88
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 49.82 min. Items per second: 12.75


In [38]:
result_dict, _ = evaluator_test.evaluateRecommender(newRecommender)

best_MAP_test = result_dict.loc[10]["MAP"]

best_MAP_test

EvaluatorHoldout: Processed 35595 (100.0%) in 44.40 sec. Users per second: 802


0.05697943464504593

In [37]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9


In [39]:
user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(newRecommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv("submissions/slimnet_notWhole.csv", index=False)

# Let's improve our Slimnet model

Now we'll split the model in two portion, in a 80-20 manner. Then, based on the previous results, I'll repeat the optuna process with a smaller hyperspace.

In [3]:
import os
os.chdir('../')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [6]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


## Optuna new test


In [11]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

def objective_function(trial):

    params = {
        "topK" : trial.suggest_int("topK", 300, 800),  # Range ristretto
        "l1_ratio" : trial.suggest_float("l1_ratio", 1e-5, 1e-4, log=True),  # Distribuzione logaritmica
        "alpha" : trial.suggest_float("alpha", 0.001, 0.2, log=True),
    }


    MAP = 0
    recommender = SLIMElasticNetRecommender(URM_train_validation)
    recommender.fit(**params)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [12]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Slimnet_optune2.csv", index = False)

In [14]:
import optuna

sampler = optuna.samplers.TPESampler(
    multivariate=True
)
optuna_study = optuna.create_study(sampler=sampler, direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      n_jobs=4)

/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-11-29 16:21:53,254] A new study created in memory with name: no-name-1d1c2e00-4293-42ac-acde-f3295fa00dc5


SLIMElasticNetRecommender: Processed 3361 ( 8.8%) in 5.00 min. Items per second: 11.20
SLIMElasticNetRecommender: Processed 4077 (10.7%) in 5.00 min. Items per second: 13.58
SLIMElasticNetRecommender: Processed 4792 (12.6%) in 5.00 min. Items per second: 15.97
SLIMElasticNetRecommender: Processed 4893 (12.8%) in 5.00 min. Items per second: 16.30
SLIMElasticNetRecommender: Processed 6646 (17.4%) in 10.00 min. Items per second: 11.07
SLIMElasticNetRecommender: Processed 8038 (21.1%) in 10.00 min. Items per second: 13.39
SLIMElasticNetRecommender: Processed 9473 (24.8%) in 10.00 min. Items per second: 15.78
SLIMElasticNetRecommender: Processed 9669 (25.4%) in 10.00 min. Items per second: 16.11
SLIMElasticNetRecommender: Processed 10071 (26.4%) in 15.00 min. Items per second: 11.19
SLIMElasticNetRecommender: Processed 14273 (37.4%) in 15.00 min. Items per second: 15.86
SLIMElasticNetRecommender: Processed 14634 (38.4%) in 15.00 min. Items per second: 16.26
SLIMElasticNetRecommender: Proces

[I 2024-11-29 16:59:33,395] Trial 2 finished with value: 0.03244622037605513 and parameters: {'topK': 798, 'l1_ratio': 1.163097910281427e-05, 'alpha': 0.14929565196812855}. Best is trial 2 with value: 0.03244622037605513.


SLIMElasticNetRecommender: Processed 38121 (100.0%) in 38.52 min. Items per second: 16.49
EvaluatorHoldout: Processed 35595 (100.0%) in 28.72 sec. Users per second: 1239


[I 2024-11-29 17:00:53,410] Trial 3 finished with value: 0.04154749641580495 and parameters: {'topK': 729, 'l1_ratio': 8.347557758290899e-05, 'alpha': 0.027661538414329983}. Best is trial 3 with value: 0.04154749641580495.


...

/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.25295668840408325, tolerance: 0.044199999421834946
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.030349131673574448, tolerance: 0.02159999869763851
  model = cd_fast.sparse_enet_coordinate_descent(


EvaluatorHoldout: Processed 35595 (100.0%) in 27.39 sec. Users per second: 1299


[I 2024-11-30 00:31:11,415] Trial 27 finished with value: 0.05712188487325251 and parameters: {'topK': 776, 'l1_ratio': 7.995986796639301e-05, 'alpha': 0.0013911853435436088}. Best is trial 25 with value: 0.057739659351119736.


SLIMElasticNetRecommender: Processed 14764 (38.7%) in 25.01 min. Items per second: 9.84
SLIMElasticNetRecommender: Processed 13479 (35.4%) in 25.01 min. Items per second: 8.98
SLIMElasticNetRecommender: Processed 36129 (94.8%) in 1.33 hour. Items per second: 7.53


KeyboardInterrupt: 